In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
data_dir = '../'
# Week of year, calculated as the number as of EOD Saturday of that week.
# Saturday
month_day = [
    '01-04',
    '01-11',
    '01-18',
    '01-25',
    '02-01',
    '02-08',
    '02-15',
    '02-22',
    '02-29',
    '03-07',
    '03-14',
    '03-21',
    '03-28',
    '04-04',
    '04-11',
    '04-18',
    '04-25',
    '05-02',
    '05-09',
    '05-16'
]
weeks = ['2020-'+md for md in month_day]

In [3]:
# Load the list of zips and fips used in project
zip2fips = pd.read_csv("../zip2fits.csv")

### Cases and deaths by FIPS

In [4]:
''' Load NYTimes COVID cases and deaths by county data
 Source: https://github.com/nytimes/covid-19-data
Features:
cases - Number of cases at a given time (it's a running total, not new cases)
deaths - Number of daths at a given time (it's a running total, not new cases)
Could add net new cases/deaths as features.
'''
df = pd.read_csv(data_dir+"covid-19-data/us-counties.csv")
# Remove NaN fips codes
df.dropna(axis=0,subset=['fips'],inplace=True)
# Convert fips float to int, as fips_int column
df['fips_int'] = [int(fip) for fip in df['fips']]
# Sort by FIPS then date
nytimes_covid_cases = df[df['date'].isin(weeks)].sort_values(by=['fips_int','date'])


In [5]:
nytimes_covid_cases['week'] = [datetime.strptime(dat,"%Y-%m-%d").strftime('%U') for dat in nytimes_covid_cases['date']]

In [6]:
# Walk FIPS->ZIP
nytimes_covid_cases = nytimes_covid_cases.merge(zip2fips[['FIPS','ZIP']], \
                                                left_on='fips_int', \
                                                right_on='FIPS') \
                                                .drop(['fips_int', \
                                                       'fips'],axis=1)

In [7]:
# Add zeroes for missing, early weeks
a = nytimes_covid_cases['ZIP'].value_counts()
tens = a[a==10]
nines = a[a==9]
for zipp in tens.index:        
    record = nytimes_covid_cases[(nytimes_covid_cases['ZIP']==zipp) & (nytimes_covid_cases['week']==str(15))].copy()
    record['deaths']=0
    record['cases']=0
    record['week']='08'
    nytimes_covid_cases = nytimes_covid_cases.append(record)
for zipp in nines.index:        
    record9 = nytimes_covid_cases[(nytimes_covid_cases['ZIP']==zipp) & (nytimes_covid_cases['week']==str(15))].copy()
    record9['deaths']=0
    record9['cases']=0
    record9['week']='08'
    record10 = record9.copy()
    record10['week']='09'
    nytimes_covid_cases = nytimes_covid_cases.append(record9)
    nytimes_covid_cases = nytimes_covid_cases.append(record10)

### 2017 Employment by NAICS by Zipcode

In [8]:
'''
Source: https://www.census.gov/data/datasets/2017/econ/cbp/2017-cbp.html

Features:
- establishment count by naics code for each zipcode (from 2017)
- employee and establishment count for each zipcode (from 2017)
Could add: Number of establishments by biz size (headcount)
'''
# Detail file has businesses by naics code and headcount
emp_df = pd.read_csv(data_dir+"zbp17detail.txt")

# Pivot so that establishments by naics code are the columns and zip codes are the rows.
est_by_zip_naics = emp_df.pivot(index='zip',columns='naics',values='est')

# Remove /// from naics code names
cols = est_by_zip_naics.columns
cols_clean = ['num_biz_'+col.strip('/') for col in cols]
cols_dict = dict(zip(cols,cols_clean))
est_by_zip_naics.rename(columns=cols_dict,inplace=True)

# Lots of NaNs, fill with mean percent of all biz by naics column
# Calculate mean percent of all establishments for each naics column
naics_pct = [est_by_zip_naics[col].mean()/est_by_zip_naics.dropna(subset=[col])['num_biz_------'].mean() \
 for col in est_by_zip_naics]

# Create dataframe of estimated values
est_naics_num = est_by_zip_naics.copy()
for i,col in enumerate(est_naics_num):
    est_naics_num[col] = est_naics_num['num_biz_------']* naics_pct[i]

est_by_zip_naics.fillna(value=est_naics_num,inplace=True)
est_by_zip_naics.reset_index(inplace=True)

In [13]:
# Calculate means for each FIPS code, for filling missing zips
est_by_zip_fips_mean = est_by_zip_naics.merge(zip2fips,left_on='zip',right_on='ZIP').groupby('FIPS').mean()
est_by_zip_fips_mean.drop(['ZIP'],axis=1,inplace=True)

# Identify missing zips in emp_tot_output
tf = zip2fips['ZIP'].isin(est_by_zip_naics['zip'])
missing_zips = zip2fips[~tf]['ZIP'].unique()

# Fill empty zips with fips average
for i,zipp in enumerate(missing_zips):
    fipp = zip2fips[zip2fips['ZIP']==zipp]['FIPS']
    record = est_by_zip_fips_mean.loc[fipp]
    record['zip'] = zipp
    record.name = est_by_zip_naics.shape[0]+i
    est_by_zip_naics = est_by_zip_naics.append(record)

In [14]:
def naics_employment(week,zipp,naics,feature_output):
    '''
    Inputs: 
        week - int or string
        zip - int
        naics - scalar
        Return:
        naics_columns: 1x5 dataframe with the establishment count by 2,3,4,5, and 6-digit naics codes as columns'''
    if len(str(week))==1:
        week_str = '0'+str(week)
    else:
        week_str = str(week)
    naics_2 = 'num_biz_'+str(naics)[:2]+'----'
    naics_3 = 'num_biz_'+str(naics)[:3]
    naics_4 = 'num_biz_'+str(naics)[:4]
    naics_5 = 'num_biz_'+str(naics)[:5]
    naics_6 = 'num_biz_'+str(naics)[:6]
    naics_columns = feature_output[(feature_output['week']==week_str)&(feature_output['ZIP']==int(zipp))] \
    [[naics_2,naics_3,naics_4,naics_5,naics_6]]
    naics_columns.rename(columns={naics_2: "naics_2_num_biz",
                                  naics_3: "naics_3_num_biz",
                                  naics_4: "naics_4_num_biz",
                                  naics_5: "naics_5_num_biz",
                                  naics_6: "naics_6_num_biz"},
                                  inplace=True)
    return naics_columns

In [15]:
# Fill missing naics codes
missing = {33:31,32:31,45:44,49:48,92:81}
other_missing = {111:"113",
1114:"1133",
11142:"11331",
111421:"113310", 
3312:"331",
33122:"331", 
331221:"331",
482:"484",
4821:"4841",
48211:"48411",
482111:"484110",
491: "484",
4911: "4841",
49111: "48411",
491110: "484110",
921:"81----",
9211:"81----",
92111:"81----",
921110:"81----",
92119:"81----",
921190:"81----",
922:"81----",
9221:"81----",
92212:"81----",
922120:"81----",
92214:"81----",
922140:"81----",
92216:"81----",
922160:"81----",
923:"81----",
9231:"81----",
92314:"81----",
923140:"81----",
926:"81----",
9261:"81----",
92612:"81----",
926120:"81----",
928:"81----",
9281:"81----",
92812:"81----",
928120:"81----"
}

for bad,good in missing.items():
    est_by_zip_naics['num_biz_'+str(bad)+'----'] = est_by_zip_naics['num_biz_'+str(good)+'----']

for bad,good in other_missing.items():
    est_by_zip_naics['num_biz_'+str(bad)] = est_by_zip_naics['num_biz_'+str(good)]

### Other Employment dataset

In [16]:
# Total employment count and establishment count by zipcode.  Latest data is 2017.
emp_tot_df = pd.read_csv(data_dir+"zbp17totals.txt")

# Taking midpoints
emp_codes ={"A": 10, 
            "B": 80/2+20,
            "C": (150/2)+100,
            "E": (500-250)/2+250,
            "F": 500/2+500,
            "G": (2500-1000)/2+1000,
            "H": (5000-2500)/2+2500,
            "I": (10000-5000)/2+5000,
            "J": (15000/2)+10000,
            "K": (25000/2)+25000,
            "L": (50000/2)+50000,
            "M": 250000
            }

# Lookup the code employment estimates
for index, row in emp_tot_df[emp_tot_df['empflag'].notna()].iterrows():
    emp_tot_df.loc[index,'emp'] = emp_codes[row['empflag']]

# Just employent and number of establishments by zip code
emp_tot_output = emp_tot_df[['zip','emp','est']]

In [17]:
# Calculate means for each FIPS code, for filling missing zips
emp_tot_output_fips_mean = emp_tot_output.merge(zip2fips,left_on='zip',right_on='ZIP').groupby('FIPS').mean()

# Identify missing zips in emp_tot_output
# tf = [zipp not in emp_tot_output['zip'] for zipp in zip2fips['ZIP']]
tf = zip2fips['ZIP'].isin(emp_tot_output['zip'])
missing_zips = zip2fips[~tf]['ZIP'].unique()
missing_zips

# Fill empty zips with fips average
for zipp in missing_zips:
    record = emp_tot_output.iloc[0].copy()
    record['zip'] = zipp
    fipp = zip2fips[zip2fips['ZIP']==zipp]['FIPS']
    record['emp'] = emp_tot_output_fips_mean.loc[fipp]['emp']
    record['est'] = emp_tot_output_fips_mean.loc[fipp]['est']
    emp_tot_output = emp_tot_output.append(record)

### Demographics by FIPS Code

In [18]:
''' Source: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html 
            (datasets section at bottom of page)
    Features (by FIPS:
    POPESTIMATE2019 - population estimate for 2019 based on 2010 census, projected growth
    Pop_pct_chg_2019 - Projected pct change in population from July 2018 - July 2019            
    Possible other features: birth rate, death rate, migration rate.  Decomposition of pop growth rate
'''

' Source: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html \n            (datasets section at bottom of page)\n    Features (by FIPS:\n    POPESTIMATE2019 - population estimate for 2019 based on 2010 census, projected growth\n    Pop_pct_chg_2019 - Projected pct change in population from July 2018 - July 2019            \n    Possible other features: birth rate, death rate, migration rate.  Decomposition of pop growth rate\n'

In [19]:
demog_df = pd.read_csv(data_dir+"co-est2019-alldata.csv",encoding='latin-1')

# Combine state-county for fips code
demog_df['fips'] = demog_df['STATE']*1000+demog_df['COUNTY']
demog_df['Pop_pct_chg_2019'] = demog_df['NPOPCHG_2019']/demog_df['POPESTIMATE2019']
demog_output_df = demog_df[['fips','POPESTIMATE2019','Pop_pct_chg_2019']]

In [20]:
demog_output_df = demog_output_df.merge(zip2fips[['FIPS','ZIP']], \
                                                left_on='fips', \
                                                right_on='FIPS') \
                                                .drop(['fips'],axis=1)

### Weather Data

In [21]:
'''
Sources: https://console.cloud.google.com/bigquery?project=my-project-2353-182914&folder=&organizationId=&j=bq:US:bquxjob_451482fc_1720b3f21bd&page=queryresults
and to map zips to stations: https://get-weather-data.readthedocs.io/en/latest/zip2ws.html
'''
pass

In [22]:
# Load zip lat-long file
latlonmap = pd.read_csv(data_dir+"us-zip-code-latitude-and-longitude.csv")

# Load and combine the raw stataion weatherdatasets
w1 = pd.read_csv(data_dir+"2151907.csv")
w2 = pd.read_csv(data_dir+"2151908.csv")
w3 = pd.read_csv(data_dir+"2151909.csv")
drop_cols = ['PSUN','TSUN','WT05','WT09','WT07']
w1 = w1.drop(drop_cols,axis=1)
w2 = w2.drop(drop_cols[:4],axis=1)
weather_stations_df = pd.concat([w1,w2,w3],axis=0)

# Drop stations without temp readings
weather_stations_df.dropna(axis=0,subset=['TMIN','TMAX'],inplace=True)

In [23]:
# Finding the closest station
# Create df of stations and their lat-long for finding closest
stationlatlon = weather_stations_df.groupby('STATION').max()[['LATITUDE','LONGITUDE']]

# Vlookup lat lon for each zip code, for finding closest station
ziplatlon = zip2fips.merge(latlonmap,left_on='ZIP',right_on='Zip')[['ZIP','FIPS','Latitude','Longitude']]    

# Calculate and map the closest station for each zip code
closest_station=[]
for index,row in ziplatlon.iterrows():

    st_dist = []
    for st_index,station in stationlatlon.iterrows():
        deglen = 110.25
        x = station['LATITUDE']-row['Latitude']
        y = (station['LONGITUDE']-row['Longitude'])*np.cos(row['Latitude']*np.pi/180)
        dist = deglen*np.sqrt(x**2+y**2)
        st_dist.append([station.name,dist])
    st_dist_df = pd.DataFrame(st_dist)
    c_s = st_dist_df.loc[st_dist_df[1].idxmin()][0]
    closest_station.append(c_s)
ziplatlon['closest_station']=closest_station

In [24]:
# Combine station weather data and zip list
weather_df = ziplatlon.merge(weather_stations_df,left_on="closest_station",right_on="STATION")

In [25]:
# Map days to their weeks
weather_df['week'] = [datetime.strptime(dat,"%Y-%m-%d").strftime('%U') for dat in weather_df['DATE']]
# fill nan's for weather fields
weather_df[['SNOW','PRCP','WT01','WT02','WT03','WT04','WT06','WT08','WT11']]= \
weather_df[['SNOW','PRCP','WT01','WT02','WT03','WT04','WT06','WT08','WT11']].fillna(0)

#Group by week and create agg features: min temp ,max temp, sum of precip, 
weather_shaped = weather_df.groupby(['ZIP','week']).agg({'TMIN': 'min', 
                                'TMAX': 'max',
                                'PRCP':'sum',
                                'SNOW':'sum',
                                'WT01':'sum', 
                                'WT02':'sum',
                                'WT03':'sum', 
                                'WT04':'sum', 
                                'WT06':'sum', 
                                'WT08':'sum', 
                                'WT11':'sum'})

# Reset index so that ZIP and week are columns
weather_shaped.reset_index(inplace=True)

## Combine data!

In [26]:
est_by_zip_naics.head()

naics,zip,num_biz_------,num_biz_11----,num_biz_113,num_biz_1131,num_biz_11311,num_biz_113110,num_biz_1133,num_biz_11331,num_biz_113310,...,num_biz_92314,num_biz_923140,num_biz_926,num_biz_9261,num_biz_92612,num_biz_926120,num_biz_928,num_biz_9281,num_biz_92812,num_biz_928120
0,1001,473.0,6.241040,10.123215,3.277136,3.277136,3.277136,10.556008,10.556008,10.556008,...,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
1,1002,545.0,7.191050,11.664170,3.775982,3.775982,3.775982,12.162842,12.162842,12.162842,...,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000
2,1003,21.0,0.277086,0.449445,0.145497,0.145497,0.145497,0.468660,0.468660,0.468660,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
3,1004,12.0,0.158335,0.256826,0.083141,0.083141,0.083141,0.267806,0.267806,0.267806,...,1.171485,1.171485,1.171485,1.171485,1.171485,1.171485,1.171485,1.171485,1.171485,1.171485
4,1005,94.0,1.240291,2.011802,0.651270,0.651270,0.651270,2.097811,2.097811,2.097811,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [27]:
demog_output_df.head()

,POPESTIMATE2019,Pop_pct_chg_2019,FIPS,ZIP
0,759297,0.005783,13089,30307
1,759297,0.005783,13089,30315
2,759297,0.005783,13089,30319
3,759297,0.005783,13089,30360
4,759297,0.005783,13089,30306


In [28]:
emp_tot_output.head()

,zip,emp,est
0,1001.0,9891.0,473.0
1,1002.0,8482.0,545.0
2,1003.0,337.0,21.0
3,1004.0,213.0,12.0
4,1005.0,1248.0,94.0


In [29]:
# Combine non-date datasets
def combine_non_date(demog_output_df,emp_tot_output,est_by_zip_naics):
    a = demog_output_df.merge(emp_tot_output,how='left',left_on='ZIP',right_on='zip')
    b = a.merge(est_by_zip_naics,how='left',left_on='ZIP',right_on='zip')
    return b
non_date_features = combine_non_date(demog_output_df,emp_tot_output,est_by_zip_naics)

In [30]:
# Combine date datasets
date_features = weather_shaped.merge(nytimes_covid_cases,how='right',on=['ZIP','week'])
date_features.head()

,ZIP,week,TMIN,TMAX,PRCP,SNOW,WT01,WT02,WT03,WT04,WT06,WT08,WT11,date,county,state,cases,deaths,FIPS
0,10001,08,25.0,62.0,0.54,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,2020-04-18,New York City,New York,0,0,36061
1,10001,09,25.0,59.0,0.63,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-07,New York City,New York,12,0,36061
2,10001,10,35.0,72.0,0.29,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-14,New York City,New York,269,1,36061
3,10001,11,33.0,77.0,1.09,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,2020-03-21,New York City,New York,6226,75,36061
4,10001,12,33.0,69.0,1.62,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-03-28,New York City,New York,30919,825,36061


In [31]:
# Map non-date features onto date_features
feature_output = date_features.merge(non_date_features,how='left',on='ZIP')

In [ ]:
# create csv file
feature_output.to_csv("external_features.csv")